In [ ]:
import pandas as pd

In [ ]:
dost_df = pd.read_excel('/content/корпус НА 1207.xlsx', index_col=0)

In [ ]:
df = pd.DataFrame({'text': dost_df['text_trans']})

Для аннотирования используем морфологический анализатор stanza. Он работает долго: необходимо будет подождать около 15-20 минут.


In [ ]:
!pip install stanza
import stanza
stanza.download("ja")
nlp_stanza = stanza.Pipeline(lang="ja", processors="tokenize, pos, lemma, depparse, ner")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 41.7 MB/s eta 0:00:00


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: ja (Japanese) ...


INFO:stanza:Downloaded file to /root/stanza_resources/ja/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: ja (Japanese):
| Processor | Package      |
----------------------------
| tokenize  | gsd          |
| pos       | gsd_charlm   |
| lemma     | gsd_nocharlm |
| depparse  | gsd_charlm   |
| ner       | gsd          |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
import re

1. Препроцессинг.
Вытаскиваем все пробелы, знаки препинания, числа. Станза на входе принимают строку. Приводим к необходимому формату.  Не забываем добавлять точки в конце каждого предложения (иначе может неправильно аннотировать).

In [ ]:
df_a1 = pd.DataFrame.to_string(df)
df_a1 = re.sub(r'\n', ' ', df_a1)
df_a1 = re.sub(r'[A-Za-z]', '', df_a1)
df_a1 = re.sub(r'\d', '', df_a1)
df_a1 = re.sub(r'\.', '', df_a1)
df_a1 = re.sub(r' (?= .)', '。', df_a1)
df_a1 = re.sub(r'。+', '。', df_a1)
df_a1 = re.sub(r'。 ', '', df_a1, count=1)
df_a1

'さっきの印象が心を去らぬ。 さっきの印象が、まだほとんど残っていたが、。 先ほどの印象はあれからずっと彼の心を去らなかったので、。 さっきの印象はほとんどそのままに残っていた。 さっき受けた印象がなかなか心から離れず、。 ということも、不快なショックを与えた。 頼まれたことを思い、ガーニャがアグラーヤにやる手紙のことを思うと、不愉快な気持がしてならなかった。 …と考えることも心に不愉快なショックを与えた。 考え自体も、また不快な驚きだった。 依頼は…こと自体に、不快な驚きを覚えていた。 なにかしら重苦しい不快なものが、公爵の胸を刺したように感じられた。 何かしら、重苦しい、不愉快なものがあたかも公爵の心をちくりと刺したように思われた。 何かしら重苦しい不愉快なものが、公爵の胸をチクリと刺したように思われた。 公爵は何か重苦しい、不快なものに傷つけられたような気がした。 公爵は、何かしら重苦しく不快なものに傷つけられたような感じがした。 彼らはかならず自分たち両人をＰＶ線の三等車に向かいあってすわらせた運命の奇怪さに驚いたであろう。 自分たちが不思議な偶然によって、ペテルブルグ・ワルシャワ線の三等車に膝をつき合わして坐っている。 二人は自分たちを…列車に向い合せた運命の偶然に。 きっと奇しくも自分たちをこの列車に向かい合わせに座らせた偶然な不思議さ。 この二人が…列車に乗り合わせた偶然のいたずらに。 どんな興味がこの連中を支配している。 この連中がどんな興味に動かされているの。 どんな興味がこの連中を支配している。 こういう情報通氏がいったいどのような関心に突き動かされているのか。 連中がどの関心に突き動かされているか。 役人は妙なしかめ面をし、思わずぶるぶるっと身震いしながら。 役人はしかめづらをして、身震いさえもしながら、。 ぶるぶるっと身震いさえもしながら、。 役人は恐怖に顔を歪め、おまけに体中ぶるぶる震わせていた。 役人は顔をしかめた。体に震えまで来ているようだった。 あなたのお邪魔をしたと思うと、それがどうも心にとがめて。 と思うと、恐縮の至りなんですが。 と思うと、気がとがめてなりませんが。 と思っただけで、もう気が咎めて。 と思うだけで、もう気がとがめます。 彼は特病の発作が頻繁なため、ほとんど白痴同然になってしまった。 彼は持病の発作がたびたび起こる

In [ ]:
doc = nlp_stanza(df_a1)

Если оставить всё как есть, то не будет в таблице деления на предложения. Можно написать цикл, чтобы добавить id.

In [ ]:
list_of_rows = []
counter = 0
for sentence in doc.sentences:
    counter += 1
    for word in sentence.words:
        list_of_rows.append([counter, word.id, word.text, word.upos, word.deprel, word.head, sentence.words[word.head-1].text])

df_sentence = pd.DataFrame(list_of_rows, columns=['sent_id', 'id', 'token', 'pos', 'synt_tag', 'head_id', 'head_tok'])
df_sentence[:10]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
0,1,1,さっき,NOUN,nmod,3,印象
1,1,2,の,ADP,case,1,さっき
2,1,3,印象,NOUN,nsubj,7,去らぬ
3,1,4,が,ADP,case,3,印象
4,1,5,心,NOUN,obj,7,去らぬ
5,1,6,を,ADP,case,5,心
6,1,7,去らぬ,VERB,root,0,。
7,1,8,。,PUNCT,punct,7,去らぬ
8,2,1,さっき,NOUN,nmod,3,印象
9,2,2,の,ADP,case,1,さっき


Вытаскиваем подлежащее

In [ ]:
nsubj = df_sentence.loc[(df_sentence['synt_tag'] == 'nsubj')]
nsubj[:30]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
2,1,3,印象,NOUN,nsubj,7,去らぬ
10,2,3,印象,NOUN,nsubj,8,残っ
25,3,4,印象,NOUN,nsubj,13,去ら
43,4,3,印象,NOUN,nsubj,9,残っ
57,5,4,印象,NOUN,nsubj,9,離れ
68,6,3,こと,NOUN,nsubj,10,与え
85,7,8,ガーニャ,PROPN,nsubj,12,やる
100,7,23,気持,NOUN,nsubj,25,し
111,8,4,こと,NOUN,nsubj,13,与え
124,9,2,自体,NOUN,nsubj,8,驚き


Можно задавать более сложное условие. Здесь мы попросим его найти все глаголы в страдательном залоге через связь 'aux' и непосредственно токен れ

In [ ]:
aux = df_sentence.loc[(df_sentence['synt_tag'] == 'aux')&(df_sentence['token'] == 'れ')]
aux[:30]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
79,7,2,れ,AUX,aux,1,頼ま
194,12,24,れ,AUX,aux,23,思わ
218,13,22,れ,AUX,aux,21,思わ
397,22,9,れ,AUX,aux,7,動か
428,24,16,れ,AUX,aux,14,動か
442,25,9,れ,AUX,aux,7,動か
738,45,11,れ,AUX,aux,9,打ちのめ
751,46,6,れ,AUX,aux,5,殺さ
824,51,7,れ,AUX,aux,6,覚まさ
918,58,4,れ,AUX,aux,3,うた


In [ ]:
comp = df_sentence.loc[(df_sentence['synt_tag'] == 'compound')]
comp[:30]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
97,7,20,不,NOUN,compound,21,愉快
115,8,8,不,NOUN,compound,9,愉快
136,10,3,…,NOUN,compound,5,自体
137,10,4,こと,NOUN,compound,5,自体
177,12,7,不,NOUN,compound,8,愉快
201,13,5,不,NOUN,compound,6,愉快
231,14,11,傷,NOUN,compound,12,つけ
254,15,13,傷,NOUN,compound,14,つけ
271,16,7,自分,NOUN,compound,9,両人
272,16,8,たち,NOUN,compound,9,両人


In [ ]:
propn = df_sentence.loc[(df_sentence['pos'] == 'PROPN')]
propn

,sent_id,id,token,pos,synt_tag,head_id,head_tok
85,7,8,ガーニャ,PROPN,nsubj,12,やる
87,7,10,アグラーヤ,PROPN,obl,12,やる
308,17,11,ペテルブルグ,PROPN,compound,14,線
310,17,13,ワルシャワ,PROPN,compound,14,線
416,24,4,通,PROPN,compound,5,氏
...,...,...,...,...,...,...,...
46796,3099,3,吹,PROPN,compound,4,雪
46963,3113,1,リュドミラ,PROPN,nsubj,9,染め
47000,3116,4,エラーギン,PROPN,obj,6,狼狽
47768,3171,1,トロヤ,PROPN,nsubj,16,ある


Попробуем вытащить винительный падеж. В качестве head_tok будет предыдущее слово, поэтому можно встретить たち, こと и т.п.

In [ ]:
acc = df_sentence.loc[(df_sentence['synt_tag'] == 'case')&(df_sentence['token'] == 'を')]
acc

,sent_id,id,token,pos,synt_tag,head_id,head_tok
5,1,6,を,ADP,case,5,心
33,3,12,を,ADP,case,11,心
74,6,9,を,ADP,case,8,ショック
82,7,5,を,ADP,case,4,こと
93,7,16,を,ADP,case,15,こと
...,...,...,...,...,...,...,...
48172,3197,8,を,ADP,case,7,心臓
48179,3198,4,を,ADP,case,3,人間
48190,3199,5,を,ADP,case,4,愛
48304,3203,12,を,ADP,case,11,息


In [ ]:
!pip install docx2txt

In [ ]:
import docx2txt

In [ ]:
speech = docx2txt.process('/content/安倍所信発明平成30年10月24日.docx')

In [ ]:
sp = nlp_stanza(speech)

In [ ]:
list_of_rows = []
counter = 0
for sentence in sp.sentences:
    counter += 1
    for word in sentence.words:
        list_of_rows.append([counter, word.id, word.text, word.upos, word.deprel, word.head, sentence.words[word.head-1].text])

sp_sentence = pd.DataFrame(list_of_rows, columns=['sent_id', 'id', 'token', 'pos', 'synt_tag', 'head_id', 'head_tok'])
sp_sentence[:10]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
0,1,1,平成,PROPN,compound,7,日
1,1,2,30,NUM,compound,7,日
2,1,3,年,NOUN,compound,7,日
3,1,4,10,NUM,compound,7,日
4,1,5,月,NOUN,compound,7,日
5,1,6,24,NUM,nummod,7,日
6,1,7,日,NOUN,root,0,日
7,2,1,第,NOUN,compound,7,国会
8,2,2,百,NUM,compound,7,国会
9,2,3,九,NUM,compound,7,国会


In [ ]:
acc_sp = sp_sentence.loc[(sp_sentence['synt_tag'] == 'case')&(sp_sentence['token'] == 'を')]
acc_sp

,sent_id,id,token,pos,synt_tag,head_id,head_tok
46,4,20,を,ADP,case,19,受賞
72,5,21,を,ADP,case,20,光
92,7,3,を,ADP,case,2,定説
108,7,19,を,ADP,case,18,偉業
169,9,17,を,ADP,case,16,進歩
...,...,...,...,...,...,...,...
3841,164,10,を,ADP,case,9,国創り
3863,165,8,を,ADP,case,7,ところ
3903,166,14,を,ADP,case,13,奪還
3984,170,10,を,ADP,case,9,身


In [ ]:
nsubj_sp = sp_sentence.loc[(sp_sentence['synt_tag'] == 'nsubj')]
nsubj_sp[:30]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
57,5,6,成果,NOUN,nsubj,22,もたらし
99,7,10,世界,NOUN,nsubj,12,広がる
137,8,1,世界,NOUN,nsubj,11,変化
164,9,12,知能,NOUN,nsubj,18,遂げ
181,9,29,（りょう,NOUN,nsubj,31,）
194,10,4,データ,NOUN,nsubj,11,駆け巡り
221,11,8,年,NOUN,nsubj,10,あれ
226,11,13,世界,NOUN,nsubj,29,ない
230,11,17,たち,NOUN,nsubj,22,でき
233,11,20,想像,NOUN,nsubj,22,でき


In [ ]:
#Достоевский
attempt = df_sentence.loc[(df_sentence['synt_tag'] == 'root') | ((df_sentence['synt_tag'] == 'case')&(df_sentence['token'] == 'を')) | (df_sentence['synt_tag'] == 'obj')]
attempt[:10]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
6,1,7,日,NOUN,root,0,日
24,2,18,9,NUM,root,0,9
31,3,7,%,NOUN,root,0,%
33,4,2,はじめ,NOUN,root,0,に
53,5,19,受賞,NOUN,obj,23,申し上げ
54,5,20,を,ADP,case,19,受賞
57,5,23,申し上げ,VERB,root,0,。
79,6,20,光,NOUN,obj,22,もたらし
80,6,21,を,ADP,case,20,光
81,6,22,もたらし,VERB,root,0,。


In [ ]:
auxsp = sp_sentence.loc[(sp_sentence['synt_tag'] == 'aux')&(sp_sentence['token'] == 'れ')]
auxsp[:30]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
425,22,5,れ,AUX,aux,4,なら
802,37,18,れ,AUX,aux,16,解除
961,43,20,れ,AUX,aux,17,抱い
2328,104,6,れ,AUX,aux,4,来日
2344,104,22,れ,AUX,aux,21,なら
2537,112,17,れ,AUX,aux,16,残さ
2731,119,15,れ,AUX,aux,13,締結
3160,136,7,れ,AUX,aux,6,行わ
3175,136,22,れ,AUX,aux,21,課さ
3307,142,17,れ,AUX,aux,15,撤廃


In [ ]:
attempt_sp = sp_sentence.loc[(sp_sentence['synt_tag'] == 'root') | ((sp_sentence['synt_tag'] == 'case')&(sp_sentence['token'] == 'を')) | (sp_sentence['synt_tag'] == 'obj')]
attempt_sp[:10]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
6,1,7,日,NOUN,root,0,日
23,2,17,演説,NOUN,root,0,演説
25,3,2,はじめ,NOUN,root,0,に
45,4,19,受賞,NOUN,obj,23,申し上げ
46,4,20,を,ADP,case,19,受賞
49,4,23,申し上げ,VERB,root,0,。
71,5,20,光,NOUN,obj,22,もたらし
72,5,21,を,ADP,case,20,光
73,5,22,もたらし,VERB,root,0,。
85,6,9,誇り,NOUN,root,0,。


In [ ]:
grouped_sp = attempt_sp.groupby(['sent_id']).sum()
grouped_sp = grouped_sp.loc[(grouped_sp['synt_tag'] != 'root') & (grouped_sp['synt_tag'] != 'case') & (grouped_sp['synt_tag'] != 'nobj')]

In [ ]:
grouped_sp

,id,token,pos,synt_tag,head_id,head_tok
sent_id,,,,,,
4,62,受賞を申し上げ,NOUNADPVERB,objcaseroot,42,申し上げ受賞。
5,63,光をもたらし,NOUNADPVERB,objcaseroot,42,もたらし光。
7,84,定説を偉業をアプローチ,NOUNADPNOUNADPNOUN,objcaseobjcaseroot,44,覆す定説もたらし偉業。
9,120,進歩を能力をし,NOUNADPNOUNADPVERB,objcaseobjcaseroot,91,遂げ進歩）能力。
10,66,世界を価値をなり,NOUNADPNOUNADPVERB,objcaseobjcaseroot,50,駆け巡り世界生み出す価値。
...,...,...,...,...,...,...
164,39,国創りを語っ,NOUNADPVERB,objcaseroot,22,進める国創り。
165,47,ところを挑戦,NOUNADPVERB,objcaseroot,16,考察ところ。
166,63,奪還をあり,NOUNADPVERB,objcaseroot,28,成し奪還。


In [ ]:
grouped_sp = attempt_sp.groupby(['sent_id']).sum()
grouped_sp = grouped_sp.loc[(grouped_sp['synt_tag'] != 'root') & (grouped_sp['synt_tag'] != 'case') & (grouped_sp['synt_tag'] != 'nobj')]

#grouped_sp['sent_id'] = grouped_sp['sent_id'].astype(int)
#grouped_sp = grouped_sp.sort_values(by=['sent_id'], ignore_index = True)

tadoshi_final_sp = grouped_sp.drop(columns=['id', 'pos', 'synt_tag', 'head_id', 'head_tok'], axis = 1)

#Сохраняем в отдельную таблицу
tadoshi_final_sp.to_excel('tadoshi_expression_sp.xlsx', index = False)


In [ ]:
speechAbe1 = docx2txt.process('/content/安倍2006所信発明9月29日.docx')

In [ ]:
spAbe1 = nlp_stanza(speechAbe1)

In [ ]:
list_of_rows = []
counter = 0
for sentence in spAbe1.sentences:
    counter += 1
    for word in sentence.words:
        list_of_rows.append([counter, word.id, word.text, word.upos, word.deprel, word.head, sentence.words[word.head-1].text])

spAbe1_sentence = pd.DataFrame(list_of_rows, columns=['sent_id', 'id', 'token', 'pos', 'synt_tag', 'head_id', 'head_tok'])
spAbe1_sentence[:10]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
0,1,1,データベース,NOUN,nmod,3,世界
1,1,2,「,PUNCT,punct,3,世界
2,1,3,世界,NOUN,nmod,5,日本
3,1,4,と,ADP,case,3,世界
4,1,5,日本,PROPN,nmod,11,明彦
5,1,6,」,PUNCT,punct,5,日本
6,1,7,（,PUNCT,punct,11,明彦
7,1,8,代表,NOUN,compound,11,明彦
8,1,9,：,PUNCT,punct,11,明彦
9,1,10,田中,PROPN,compound,11,明彦


In [ ]:
acc_spAbe1 = spAbe1_sentence.loc[(spAbe1_sentence['synt_tag'] == 'case')&(spAbe1_sentence['token'] == 'を')]
acc_spAbe1

,sent_id,id,token,pos,synt_tag,head_id,head_tok
126,6,5,を,ADP,case,4,時期
154,6,33,を,ADP,case,32,国政
157,6,36,を,ADP,case,35,重責
175,7,6,を,ADP,case,5,期待
184,7,15,を,ADP,case,14,身命
...,...,...,...,...,...,...,...
5142,149,31,を,ADP,case,30,日本
5151,149,40,を,ADP,case,39,尊敬
5162,149,51,を,ADP,case,50,誇り
5184,149,73,を,ADP,case,72,全霊


In [ ]:
nsubj_spAbe1 = spAbe1_sentence.loc[(spAbe1_sentence['synt_tag'] == 'nsubj')]
nsubj_spAbe1[:30]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
109,5,4,私,PRON,nsubj,11,任命
122,6,1,日本,PROPN,nsubj,6,乗り越え
201,8,8,私,PRON,nsubj,33,いたし
243,9,15,つもり,NOUN,nsubj,18,あり
303,11,1,みんな,NOUN,nsubj,3,参加
373,12,34,脱却,NOUN,nsubj,38,入る
382,12,43,成果,NOUN,nsubj,45,あらわれ
390,12,51,展望,NOUN,nsubj,53,開け
401,13,4,減少,NOUN,nsubj,10,なる
427,13,30,組,NOUN,nsubj,33,化


In [ ]:
auxspAbe1 = spAbe1_sentence.loc[(spAbe1_sentence['synt_tag'] == 'aux')&(spAbe1_sentence['token'] == 'れ')]
auxspAbe1[:30]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
118,5,13,れ,AUX,aux,11,任命
321,11,19,れ,AUX,aux,18,開か
481,14,12,れ,AUX,aux,11,失わ
610,17,31,れ,AUX,aux,30,開か
714,22,10,れ,AUX,aux,8,信頼
718,22,14,れ,AUX,aux,12,尊敬
860,25,4,れ,AUX,aux,3,開か
1263,36,15,れ,AUX,aux,14,報わ
3364,96,32,れ,AUX,aux,30,指摘
3704,107,46,れ,AUX,aux,44,採択


In [ ]:
attempt_spAbe1 = spAbe1_sentence.loc[(spAbe1_sentence['synt_tag'] == 'root') | ((spAbe1_sentence['synt_tag'] == 'case')&(spAbe1_sentence['token'] == 'を')) | (spAbe1_sentence['synt_tag'] == 'obj')]
attempt_spAbe1[:10]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
10,1,11,明彦,PROPN,root,0,）
17,2,6,データベース,NOUN,root,0,データベース
29,3,12,所,NOUN,root,0,所
72,4,43,大臣,NOUN,root,0,文]
116,5,11,任命,VERB,root,0,。
125,6,4,時期,NOUN,obj,6,乗り越え
126,6,5,を,ADP,case,4,時期
153,6,32,国政,NOUN,obj,34,預かる
154,6,33,を,ADP,case,32,国政
156,6,35,重責,NOUN,obj,37,与え


In [ ]:
grouped_spAbe1 = attempt_spAbe1.groupby(['sent_id']).sum()
grouped_spAbe1 = grouped_spAbe1.loc[(grouped_spAbe1['synt_tag'] != 'root') & (grouped_spAbe1['synt_tag'] != 'case') & (grouped_spAbe1['synt_tag'] != 'nobj')]

In [ ]:
grouped_spAbe1

,id,token,pos,synt_tag,head_id,head_tok
sent_id,,,,,,
6,191,時期を国政を重責を思い,NOUNADPNOUNADPNOUNADPNOUN,objcaseobjcaseobjcaseroot,148,乗り越え時期預かる国政与え重責。
7,60,期待を身命を取り組ん,NOUNADPNOUNADPVERB,objcaseobjcaseroot,46,受け期待賭し身命。
8,73,国政を姿勢をいたし,NOUNADPNOUNADPVERB,objcaseobjcaseroot,55,遂行国政いたし姿勢。
9,43,政治をあり,NOUNADPVERB,objcaseroot,26,行う政治。
10,257,家族をふるさとを未来を政治を行っ,NOUNADPNOUNADPNOUNADPNOUNADPVERB,objcaseobjcaseobjcaseobjcaseroot,213,愛し家族しふるさと信じ未来行っ政治。
...,...,...,...,...,...,...
145,145,すべてを述べ,NOUNADPVERB,objcaseroot,90,保っすべて。
146,95,日本を美徳を信じ,PROPNADPNOUNADPVERB,objcaseobjcaseroot,57,保ち日本保ち美徳。
148,66,環境を責任,NOUNADPNOUN,objcaseroot,40,つくる環境。


In [ ]:
grouped_spAbe1 = attempt_spAbe1.groupby(['sent_id']).sum()
grouped_spAbe1 = grouped_spAbe1.loc[(grouped_spAbe1['synt_tag'] != 'root') & (grouped_spAbe1['synt_tag'] != 'case') & (grouped_spAbe1['synt_tag'] != 'nobj')]

#grouped_sp['sent_id'] = grouped_sp['sent_id'].astype(int)
#grouped_sp = grouped_sp.sort_values(by=['sent_id'], ignore_index = True)

tadoshi_final_spAbe1 = grouped_spAbe1.drop(columns=['id', 'pos', 'synt_tag', 'head_id', 'head_tok'], axis = 1)

#Сохраняем в отдельную таблицу
tadoshi_final_spAbe1.to_excel('tadoshi_expression_spAbe1.xlsx', index = False)


In [ ]:
speechAbe07Hoshin = docx2txt.process('/content/安倍2007内閣総理大臣方針表明演説１６６.docx')

In [ ]:
spAbe07Hoshin = nlp_stanza(speechAbe07Hoshin)

In [ ]:
list_of_rows = []
counter = 0
for sentence in spAbe07Hoshin.sentences:
    counter += 1
    for word in sentence.words:
        list_of_rows.append([counter, word.id, word.text, word.upos, word.deprel, word.head, sentence.words[word.head-1].text])

spAbe07Hoshin_sentence = pd.DataFrame(list_of_rows, columns=['sent_id', 'id', 'token', 'pos', 'synt_tag', 'head_id', 'head_tok'])
spAbe07Hoshin_sentence[:10]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
0,1,1,第,NOUN,compound,4,国会
1,1,2,１６６,NOUN,compound,4,国会
2,1,3,回,NOUN,compound,4,国会
3,1,4,国会,NOUN,nmod,14,演説
4,1,5,に,ADP,case,4,国会
5,1,6,おけ,VERB,fixed,5,に
6,1,7,る,AUX,fixed,5,に
7,1,8,安倍,PROPN,compound,14,演説
8,1,9,内閣,NOUN,compound,14,演説
9,1,10,総理,NOUN,compound,14,演説


In [ ]:
acc_spAbe07Hoshin = spAbe07Hoshin_sentence.loc[(spAbe07Hoshin_sentence['synt_tag'] == 'case')&(spAbe07Hoshin_sentence['token'] == 'を')]
acc_spAbe07Hoshin

,sent_id,id,token,pos,synt_tag,head_id,head_tok
53,3,33,を,ADP,case,32,尊敬
65,4,4,を,ADP,case,3,誇り
86,4,25,を,ADP,case,24,精神
105,4,44,を,ADP,case,43,こと
141,5,25,を,ADP,case,24,感
...,...,...,...,...,...,...,...
5905,161,4,を,ADP,case,3,こと
5924,161,23,を,ADP,case,22,日本
5942,162,7,を,ADP,case,6,性
5948,162,13,を,ADP,case,12,未来


In [ ]:
nsubj_spAbe07Hoshin = spAbe07Hoshin_sentence.loc[(spAbe07Hoshin_sentence['synt_tag'] == 'nsubj')]
nsubj_spAbe07Hoshin[:30]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
27,3,7,私,PRON,nsubj,12,就任
48,3,28,人々,NOUN,nsubj,34,抱き
59,3,39,世代,NOUN,nsubj,41,自
149,5,33,こと,NOUN,nsubj,35,重要
178,7,1,私,PRON,nsubj,26,考え
222,8,17,方,NOUN,nsubj,19,築き上げ
277,9,33,多く,NOUN,nsubj,48,なっ
295,9,51,こと,NOUN,nsubj,55,明らか
302,10,1,我々,PRON,nsubj,3,直面
310,10,9,変化,NOUN,nsubj,50,もの


In [ ]:
auxspAbe07Hoshin = spAbe07Hoshin_sentence.loc[(spAbe07Hoshin_sentence['synt_tag'] == 'aux')&(spAbe07Hoshin_sentence['token'] == 'れ')]
auxspAbe07Hoshin[:30]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
94,4,33,れ,AUX,aux,32,開か
338,10,37,れ,AUX,aux,36,やさ
1586,38,9,れ,AUX,aux,7,リタイア
1763,43,16,れ,AUX,aux,15,置か
2841,74,3,れ,AUX,aux,1,改正
2895,75,29,れ,AUX,aux,27,指摘
3149,80,39,れ,AUX,aux,37,指摘
3286,84,17,れ,AUX,aux,15,改正
3569,92,11,れ,AUX,aux,9,期待
4715,125,8,れ,AUX,aux,6,禁止


In [ ]:
attempt_spAbe07Hoshin = spAbe07Hoshin_sentence.loc[(spAbe07Hoshin_sentence['synt_tag'] == 'root') | ((spAbe07Hoshin_sentence['synt_tag'] == 'case')&(spAbe07Hoshin_sentence['token'] == 'を')) | (spAbe07Hoshin_sentence['synt_tag'] == 'obj')]
attempt_spAbe07Hoshin[:10]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
13,1,14,演説,NOUN,root,0,演説
20,2,7,日,NOUN,root,0,日
52,3,32,尊敬,NOUN,obj,34,抱き
53,3,33,を,ADP,case,32,尊敬
61,3,41,自,NOUN,root,0,自
64,4,3,誇り,NOUN,obj,5,持つ
65,4,4,を,ADP,case,3,誇り
85,4,24,精神,NOUN,obj,28,する
86,4,25,を,ADP,case,24,精神
104,4,43,こと,NOUN,obj,51,示し


In [ ]:
grouped_spAbe07Hoshin = attempt_spAbe07Hoshin.groupby(['sent_id']).sum()
grouped_spAbe07Hoshin = grouped_spAbe07Hoshin.loc[(grouped_spAbe07Hoshin['synt_tag'] != 'root') & (grouped_spAbe07Hoshin['synt_tag'] != 'case') & (grouped_spAbe07Hoshin['synt_tag'] != 'nobj')]

In [ ]:
grouped_spAbe07Hoshin

,id,token,pos,synt_tag,head_id,head_tok
sent_id,,,,,,
3,106,尊敬を自,NOUNADPNOUN,objcaseroot,66,抱き尊敬自
4,194,誇りを精神をことを示し,NOUNADPNOUNADPNOUNADPVERB,objcaseobjcaseobjcaseroot,154,持つ誇りする精神示しこと。
5,144,感を結果を考え,NOUNADPNOUNADPVERB,objcaseobjcaseroot,108,持っ感出し結果。
6,43,ことを約束,NOUNADPVERB,objcaseroot,29,約束こと。
7,35,日本を考え,PROPNADPVERB,objcaseroot,26,し日本。
...,...,...,...,...,...,...
159,212,今年を全霊を覚悟,NOUNADPNOUNADPNOUN,objcaseobjcaseroot,152,位置今年傾け全霊。
160,88,事をで之を述べ,NOUNADPNOUNADPVERB,objcaseobjcaseroot,69,勤る事勤るで之。
161,78,ことを日本をいっ,NOUNADPPROPNADPVERB,objcaseobjcaseroot,54,ひることつく日本。


In [ ]:
grouped_spAbe07Hoshin = attempt_spAbe07Hoshin.groupby(['sent_id']).sum()
grouped_spAbe07Hoshin = grouped_spAbe07Hoshin.loc[(grouped_spAbe07Hoshin['synt_tag'] != 'root') & (grouped_spAbe07Hoshin['synt_tag'] != 'case') & (grouped_spAbe07Hoshin['synt_tag'] != 'nobj')]

#grouped_sp['sent_id'] = grouped_sp['sent_id'].astype(int)
#grouped_sp = grouped_sp.sort_values(by=['sent_id'], ignore_index = True)

tadoshi_final_spAbe07Hoshin = grouped_spAbe07Hoshin.drop(columns=['id', 'pos', 'synt_tag', 'head_id', 'head_tok'], axis = 1)

#Сохраняем в отдельную таблицу
tadoshi_final_spAbe07Hoshin.to_excel('tadoshi_expression_spAbe07Hoshin.xlsx', index = False)

In [ ]:
speechAbe19Hoshin = docx2txt.process('/content/安倍2019方針演説1月28日.docx')

In [ ]:
spAbe19Hoshin = nlp_stanza(speechAbe19Hoshin)

In [ ]:
list_of_rows = []
counter = 0
for sentence in spAbe19Hoshin.sentences:
    counter += 1
    for word in sentence.words:
        list_of_rows.append([counter, word.id, word.text, word.upos, word.deprel, word.head, sentence.words[word.head-1].text])

spAbe19Hoshin_sentence = pd.DataFrame(list_of_rows, columns=['sent_id', 'id', 'token', 'pos', 'synt_tag', 'head_id', 'head_tok'])
spAbe19Hoshin_sentence[:10]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
0,1,1,平成,PROPN,compound,7,日
1,1,2,31,NUM,compound,7,日
2,1,3,年,NOUN,compound,7,日
3,1,4,1,NUM,compound,7,日
4,1,5,月,NOUN,compound,7,日
5,1,6,28,NUM,nummod,7,日
6,1,7,日,NOUN,root,0,日
7,2,1,第,NOUN,compound,7,国会
8,2,2,百,NUM,compound,7,国会
9,2,3,九,NUM,compound,7,国会


In [ ]:
acc_spAbe19Hoshin = spAbe19Hoshin_sentence.loc[(spAbe19Hoshin_sentence['synt_tag'] == 'case')&(spAbe19Hoshin_sentence['token'] == 'を')]
acc_spAbe19Hoshin

,sent_id,id,token,pos,synt_tag,head_id,head_tok
33,4,7,を,ADP,case,6,演説
84,6,15,を,ADP,case,14,準備
214,10,25,を,ADP,case,24,営業
225,11,9,を,ADP,case,8,人
275,13,13,を,ADP,case,12,所
...,...,...,...,...,...,...,...
8216,321,14,を,ADP,case,13,日本
8241,322,14,を,ADP,case,13,未来
8266,323,19,を,ADP,case,18,こと
8285,324,15,を,ADP,case,14,明日


In [ ]:
nsubj_spAbe19Hoshin = spAbe19Hoshin_sentence.loc[(spAbe19Hoshin_sentence['synt_tag'] == 'nsubj')]
nsubj_spAbe19Hoshin[:30]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
48,5,8,陛下,NOUN,nsubj,11,退位
56,5,16,殿下,NOUN,nsubj,25,即位
107,7,18,災害,NOUN,nsubj,21,次い
131,8,16,姿,NOUN,nsubj,18,あり
156,9,20,陛下,NOUN,nsubj,22,焼け
165,9,29,水仙,NOUN,nsubj,48,刻ま
194,10,5,さん,NOUN,nsubj,26,再開
227,11,11,ボランティア,NOUN,nsubj,18,なり
239,12,2,水仙,NOUN,nsubj,4,置か
244,12,7,場所,NOUN,nsubj,14,変わり


In [ ]:
auxspAbe19Hoshin = spAbe19Hoshin_sentence.loc[(spAbe19Hoshin_sentence['synt_tag'] == 'aux')&(spAbe19Hoshin_sentence['token'] == 'れ')]
auxspAbe19Hoshin[:30]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
53,5,13,れ,AUX,aux,11,退位
67,5,27,れ,AUX,aux,25,即位
163,9,27,れ,AUX,aux,25,献花
185,9,49,れ,AUX,aux,48,刻ま
242,12,5,れ,AUX,aux,4,置か
422,18,16,れ,AUX,aux,14,あら
1325,55,15,れ,AUX,aux,14,奪わ
2442,98,17,れ,AUX,aux,16,行わ
3084,118,2,れ,AUX,aux,1,失わ
3192,123,32,れ,AUX,aux,31,行わ


In [ ]:
attempt_spAbe19Hoshin = spAbe19Hoshin_sentence.loc[(spAbe19Hoshin_sentence['synt_tag'] == 'root') | ((spAbe19Hoshin_sentence['synt_tag'] == 'case')&(spAbe19Hoshin_sentence['token'] == 'を')) | (spAbe1Hoshin_sentence['synt_tag'] == 'obj')]
attempt_spAbe19Hoshin[:10]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
6,1,7,日,NOUN,root,0,日
23,2,17,演説,NOUN,root,0,演説
25,3,2,はじめ,NOUN,root,0,に
33,4,7,を,ADP,case,6,演説
38,4,12,述べ,VERB,root,0,。
52,5,12,さ,AUX,aux,11,退位
64,5,24,御,NOUN,compound,25,即位
65,5,25,即位,VERB,root,0,。
84,6,15,を,ADP,case,14,準備
85,6,16,進め,VERB,root,0,。


In [ ]:
grouped_spAbe19Hoshin = attempt_spAbe19Hoshin.groupby(['sent_id']).sum()
grouped_spAbe19Hoshin = grouped_spAbe19Hoshin.loc[(grouped_spAbe19Hoshin['synt_tag'] != 'root') & (grouped_spAbe19Hoshin['synt_tag'] != 'case') & (grouped_spAbe19Hoshin['synt_tag'] != 'nobj')]

In [ ]:
grouped_spAbe19Hoshin

,id,token,pos,synt_tag,head_id,head_tok
sent_id,,,,,,
4,19,を述べ,ADPVERB,caseroot,6,演説。
5,61,さ御即位,AUXNOUNVERB,auxcompoundroot,36,退位即位。
6,31,を進め,ADPVERB,caseroot,14,準備。
7,40,」時代,PUNCTNOUN,punctroot,14,成る。
9,140,大しとの刻ま,NOUNAUXADPADPVERB,compoundauxcasecaseroot,90,震災全焼シンボル人々。
...,...,...,...,...,...,...
321,34,をし,ADPVERB,caseroot,13,日本。
322,31,を示し,ADPVERB,caseroot,13,未来。
323,39,を期待,ADPVERB,caseroot,18,こと。


In [ ]:
grouped_spAbe19Hoshin = attempt_spAbe19Hoshin.groupby(['sent_id']).sum()
grouped_spAbe19Hoshin = grouped_spAbe19Hoshin.loc[(grouped_spAbe19Hoshin['synt_tag'] != 'root') & (grouped_spAbe19Hoshin['synt_tag'] != 'case') & (grouped_spAbe19Hoshin['synt_tag'] != 'nobj')]

#grouped_sp['sent_id'] = grouped_sp['sent_id'].astype(int)
#grouped_sp = grouped_sp.sort_values(by=['sent_id'], ignore_index = True)

tadoshi_final_spAbe19Hoshin = grouped_spAbe19Hoshin.drop(columns=['id', 'pos', 'synt_tag', 'head_id', 'head_tok'], axis = 1)

#Сохраняем в отдельную таблицу
tadoshi_final_spAbe19Hoshin.to_excel('tadoshi_expression_spAbe19Hoshin.xlsx', index = False)

In [ ]:
speechHato = docx2txt.process('/content/鳩山内閣総理大臣所信表明演説.docx')

In [ ]:
spHato = nlp_stanza(speechHato)

In [ ]:
list_of_rows = []
counter = 0
for sentence in spHato.sentences:
    counter += 1
    for word in sentence.words:
        list_of_rows.append([counter, word.id, word.text, word.upos, word.deprel, word.head, sentence.words[word.head-1].text])

spHato_sentence = pd.DataFrame(list_of_rows, columns=['sent_id', 'id', 'token', 'pos', 'synt_tag', 'head_id', 'head_tok'])
spHato_sentence[:10]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
0,1,1,第,NOUN,compound,4,国会
1,1,2,１７３,NOUN,compound,4,国会
2,1,3,回,NOUN,compound,4,国会
3,1,4,国会,NOUN,nmod,14,演説
4,1,5,に,ADP,case,4,国会
5,1,6,おけ,VERB,fixed,5,に
6,1,7,る,AUX,fixed,5,に
7,1,8,鳩山,PROPN,compound,14,演説
8,1,9,内閣,NOUN,compound,14,演説
9,1,10,総理,NOUN,compound,14,演説


In [ ]:
acc_spHato = spHato_sentence.loc[(spHato_sentence['synt_tag'] == 'case')&(spHato_sentence['token'] == 'を')]
acc_spHato

,sent_id,id,token,pos,synt_tag,head_id,head_tok
56,5,10,を,ADP,case,9,指名
81,5,35,を,ADP,case,34,内閣
114,6,14,を,ADP,case,13,交代
189,8,49,を,ADP,case,48,政治
210,8,70,を,ADP,case,69,交代
...,...,...,...,...,...,...,...
8307,227,29,を,ADP,case,28,歴史
8311,227,33,を,ADP,case,32,日本
8320,227,42,を,ADP,case,41,全力
8345,228,20,を,ADP,case,19,力


In [ ]:
nsubj_spHato = spHato_sentence.loc[(spHato_sentence['synt_tag'] == 'nsubj')]
nsubj_spHato[:30]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
36,4,13,か月,NOUN,nsubj,15,経
49,5,3,私,PRON,nsubj,11,受け
90,5,44,日,NOUN,nsubj,46,経
110,6,10,さま,NOUN,nsubj,15,選択
121,7,1,これ,PRON,nsubj,18,こと
126,7,6,主義,NOUN,nsubj,8,定着
204,8,64,声,NOUN,nsubj,71,もたらし
216,8,76,私,PRON,nsubj,78,認識
237,9,14,者,NOUN,nsubj,17,一人ひとり
257,10,15,たち,NOUN,nsubj,36,取り組ん


In [ ]:
auxspHato = spHato_sentence.loc[(spHato_sentence['synt_tag'] == 'aux')&(spHato_sentence['token'] == 'れ')]
auxspHato[:30]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
117,6,17,れ,AUX,aux,15,選択
456,15,4,れ,AUX,aux,3,集まら
1129,36,10,れ,AUX,aux,9,作ら
1390,41,50,れ,AUX,aux,48,達成
1666,47,17,れ,AUX,aux,16,持た
1874,54,18,れ,AUX,aux,17,断た
2065,60,30,れ,AUX,aux,28,尊重
2825,80,26,れ,AUX,aux,24,解放
2955,84,24,れ,AUX,aux,23,頼ま
3266,98,38,れ,AUX,aux,36,構成


In [ ]:
attempt_spHato = spHato_sentence.loc[(spHato_sentence['synt_tag'] == 'root') | ((spHato_sentence['synt_tag'] == 'case')&(spHato_sentence['token'] == 'を')) | (spHato_sentence['synt_tag'] == 'obj')]
attempt_spHato[:10]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
13,1,14,演説,NOUN,root,0,演説
20,2,7,日,NOUN,root,0,日
22,3,2,はじめ,NOUN,root,0,に
42,4,19,し,VERB,root,0,。
55,5,9,指名,NOUN,obj,11,受け
56,5,10,を,ADP,case,9,指名
80,5,34,内閣,NOUN,obj,36,発足
81,5,35,を,ADP,case,34,内閣
96,5,50,し,VERB,root,0,。
113,6,13,交代,NOUN,obj,15,選択


In [ ]:
grouped_spHato = attempt_spHato.groupby(['sent_id']).sum()
grouped_spHato = grouped_spHato.loc[(grouped_spHato['synt_tag'] != 'root') & (grouped_spHato['synt_tag'] != 'case') & (grouped_spHato['synt_tag'] != 'nobj')]

In [ ]:
grouped_spHato

,id,token,pos,synt_tag,head_id,head_tok
sent_id,,,,,,
5,138,指名を内閣をし,NOUNADPNOUNADPVERB,objcaseobjcaseroot,90,受け指名発足内閣。
6,42,交代を選択,NOUNADPVERB,objcaseroot,28,選択交代。
8,314,政治を交代を認識,NOUNADPNOUNADPVERB,objcaseobjcaseroot,241,し政治もたらし交代。
10,81,歴史を取り組ん,NOUNADPVERB,objcaseroot,46,変える歴史。
12,247,期待をことを痛感,NOUNADPNOUNADPVERB,objcaseobjcaseroot,176,感ずる期待痛感こと。
...,...,...,...,...,...,...
225,319,成長を路を力を路を立っ,NOUNADPNOUNADPNOUNADPNOUNADPVERB,objcaseobjcaseobjcaseobjcaseroot,269,追い成長辿る路見いだし力見いだし路。
226,151,路を暮らしを政策を進め,NOUNADPNOUNADPNOUNADPVERB,objcaseobjcaseobjcaseroot,123,歩ん路ため暮らし推し政策。
227,277,参加を歴史を日本を全力を尽くし,NOUNADPNOUNADPPROPNADPNOUNADPVERB,objcaseobjcaseobjcaseobjcaseroot,241,得参加変え歴史導く日本尽くし全力。


In [ ]:
grouped_spHato = attempt_spHato.groupby(['sent_id']).sum()
grouped_spHato = grouped_spHato.loc[(grouped_spHato['synt_tag'] != 'root') & (grouped_spHato['synt_tag'] != 'case') & (grouped_spHato['synt_tag'] != 'nobj')]

#grouped_sp['sent_id'] = grouped_sp['sent_id'].astype(int)
#grouped_sp = grouped_sp.sort_values(by=['sent_id'], ignore_index = True)

tadoshi_final_spHato = grouped_spHato.drop(columns=['id', 'pos', 'synt_tag', 'head_id', 'head_tok'], axis = 1)

#Сохраняем в отдельную таблицу
tadoshi_final_spHato.to_excel('tadoshi_expression_spHato.xlsx', index = False)


In [ ]:
speechAso = docx2txt.process('/content/麻生内閣総理大臣所信表明演説１７０.docx')

In [ ]:
spAso = nlp_stanza(speechAso)

In [ ]:
list_of_rows = []
counter = 0
for sentence in spAso.sentences:
    counter += 1
    for word in sentence.words:
        list_of_rows.append([counter, word.id, word.text, word.upos, word.deprel, word.head, sentence.words[word.head-1].text])

spAso_sentence = pd.DataFrame(list_of_rows, columns=['sent_id', 'id', 'token', 'pos', 'synt_tag', 'head_id', 'head_tok'])
spAso_sentence[:10]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
0,1,1,第,NOUN,compound,4,国会
1,1,2,１７０,NOUN,compound,4,国会
2,1,3,回,NOUN,compound,4,国会
3,1,4,国会,NOUN,nmod,14,演説
4,1,5,に,ADP,case,4,国会
5,1,6,おけ,VERB,fixed,5,に
6,1,7,る,AUX,fixed,5,に
7,1,8,麻生,PROPN,compound,14,演説
8,1,9,内閣,NOUN,compound,14,演説
9,1,10,総理,NOUN,compound,14,演説


In [ ]:
acc_spAso = spAso_sentence.loc[(spAso_sentence['synt_tag'] == 'case')&(spAso_sentence['token'] == 'を')]
acc_spAso

,sent_id,id,token,pos,synt_tag,head_id,head_tok
50,3,30,を,ADP,case,29,璽
103,6,5,を,ADP,case,4,任命
198,9,6,を,ADP,case,5,元気
245,12,15,を,ADP,case,14,これ
254,12,24,を,ADP,case,23,飛躍
...,...,...,...,...,...,...,...
3889,206,5,を,ADP,case,4,党
3901,206,17,を,ADP,case,16,協力
3910,207,4,を,ADP,case,3,論点
3922,208,3,を,ADP,case,2,考え


In [ ]:
nsubj_spAso = spAso_sentence.loc[(spAso_sentence['synt_tag'] == 'nsubj')]
nsubj_spAso[:30]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
75,4,10,総理,NOUN,nsubj,12,列し
94,5,12,大河,NOUN,nsubj,14,あり
120,6,22,伝統,NOUN,nsubj,24,あり
145,6,47,集積,NOUN,nsubj,49,ある
161,7,9,わたし,PRON,nsubj,28,得
220,11,1,日本,PROPN,nsubj,5,あら
262,13,1,日本,PROPN,nsubj,7,なり
284,14,13,人,NOUN,nsubj,24,数々
300,14,29,わたしど,PRON,nsubj,57,知っ
334,15,2,性質,NOUN,nsubj,12,はず


In [ ]:
auxspAso = spAso_sentence.loc[(spAso_sentence['synt_tag'] == 'aux')&(spAso_sentence['token'] == 'れ')]
auxspAso[:30]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
341,15,9,れ,AUX,aux,8,受け継が
625,29,11,れ,AUX,aux,10,なさ
1697,94,9,れ,AUX,aux,8,消さ
2294,124,20,れ,AUX,aux,18,訓練
2387,127,10,れ,AUX,aux,8,駆逐
2394,127,17,れ,AUX,aux,16,救わ
3047,167,3,れ,AUX,aux,1,被災
3564,190,25,れ,AUX,aux,23,左右


In [ ]:
attempt_spAso = spAso_sentence.loc[(spAso_sentence['synt_tag'] == 'root') | ((spAso_sentence['synt_tag'] == 'case')&(spAso_sentence['token'] == 'を')) | (spAso_sentence['synt_tag'] == 'obj')]
attempt_spAso[:10]

,sent_id,id,token,pos,synt_tag,head_id,head_tok
13,1,14,演説,NOUN,root,0,演説
20,2,7,日,NOUN,root,0,日
49,3,29,璽,NOUN,obj,31,いただき
50,3,30,を,ADP,case,29,璽
61,3,41,就任,VERB,root,0,。
77,4,12,列し,VERB,root,0,。
96,5,14,あり,VERB,root,0,。
102,6,4,任命,NOUN,obj,15,続け
103,6,5,を,ADP,case,4,任命
147,6,49,ある,VERB,root,0,。


In [ ]:
grouped_spAso = attempt_spAso.groupby(['sent_id']).sum()
grouped_spAso = grouped_spAso.loc[(grouped_spAso['synt_tag'] != 'root') & (grouped_spAso['synt_tag'] != 'case') & (grouped_spAso['synt_tag'] != 'nobj')]

In [ ]:
grouped_spAso

,id,token,pos,synt_tag,head_id,head_tok
sent_id,,,,,,
3,100,璽を就任,NOUNADPVERB,objcaseroot,60,いただき璽。
6,58,任命をある,NOUNADPVERB,objcaseroot,19,続け任命。
9,33,元気をもと,NOUNADPNOUN,objcaseroot,12,失い元気。
12,103,これを飛躍を国,PRONADPNOUNADPNOUN,objcaseobjcaseroot,78,好機これ成し飛躍。
14,226,国を数々をことを知っ,NOUNADPNOUNADPNOUNADPVERB,objcaseobjcaseobjcaseroot,203,訪れ国知っ数々知っこと。
...,...,...,...,...,...,...
203,71,時間を履行を意味,NOUNADPNOUNADPVERB,objcaseobjcaseroot,52,徒費時間意味履行。
205,151,対策を財源を法案を責任,NOUNADPNOUNADPNOUNADPNOUN,objcaseobjcaseobjcaseroot,117,裏付ける対策補てん財源成立法案。
206,61,党を協力を要請,NOUNADPNOUNADPVERB,objcaseobjcaseroot,48,諸君党要請協力。


In [ ]:
grouped_spAso = attempt_spAso.groupby(['sent_id']).sum()
grouped_spAso = grouped_spAso.loc[(grouped_spAso['synt_tag'] != 'root') & (grouped_spAso['synt_tag'] != 'case') & (grouped_spAso['synt_tag'] != 'nobj')]

#grouped_sp['sent_id'] = grouped_sp['sent_id'].astype(int)
#grouped_sp = grouped_sp.sort_values(by=['sent_id'], ignore_index = True)

tadoshi_final_spAso = grouped_spAso.drop(columns=['id', 'pos', 'synt_tag', 'head_id', 'head_tok'], axis = 1)

#Сохраняем в отдельную таблицу
tadoshi_final_spAso.to_excel('tadoshi_expression_spAso.xlsx', index = False)